# Data validation
In this notebook we investigate data completeness and correctness. 

## Inspecting the data
Below we import the training data and produce some preliminary summaries.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
features = pd.read_csv('../data/training_features.csv', index_col='id')
targets = pd.read_csv('../data/training_labels.csv', index_col='id')
df = features.join(targets, how='left')
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [4]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

None

In [6]:
df.isnull().any()

amount_tsh               False
date_recorded            False
funder                    True
gps_height               False
installer                 True
longitude                False
latitude                 False
wpt_name                 False
num_private              False
basin                    False
subvillage                True
region                   False
region_code              False
district_code            False
lga                      False
ward                     False
population               False
public_meeting            True
recorded_by              False
scheme_management         True
scheme_name               True
permit                    True
construction_year        False
extraction_type          False
extraction_type_group    False
extraction_type_class    False
management               False
management_group         False
payment                  False
payment_type             False
water_quality            False
quality_group            False
quantity

In [ ]:
display(df.describe())

## Data classification
In this section we us the provided [Data Summary](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/) classify variables.

### Geographical data
There are several variables that describe the geography of waterpoints. These variables fall into two subclasses.

#### Quantitative
The variables below provide geo-spatial coordinates for the waterpoint.
* `longitude` - GPS coordinate. Valid values fall in the interval [29.6, 40.4].
* `latitude` - GPS coordinate. Valid values fall in the interval [-11.7, -0.8].
* `gps_height` - Altitude of the well. Valid values fall in the interval [0, 5895] meters.

#### Qualitative
The variables below provide a categorical description of waterpont location.
* `region` (`region_code`)  - Names (codes) for top-level administrative regions. There are 31 total [Link](https://en.wikipedia.org/wiki/Regions_of_Tanzania)
* `lga` (`district_code`) - Names (codes) for districts, which divide regions.
* `ward` - Names for wards, which divide districts.
* `subvillage` - Names for sub-villages, presumably these subdivide wards. 

### Water Table
* `basin` - Geographic water basin
* `water_quality` - The quality of the water
* `quality_group` - The quality of the water
* `quantity` - The quantity of water
* `quantity_group` - The quantity of water
* `source` - The source of the water
* `source_type` - The source of the water
* `source_class` - The source of the water

### Waterpoint
* `wpt_name` - Name of the waterpoint if there is one
* `amount_tsh` - Total static head (amount water available to waterpoint)
* `extraction_type` - The kind of extraction the waterpoint uses
* `extraction_type_group` - The kind of extraction the waterpoint uses
* `extraction_type_class` - The kind of extraction the waterpoint uses
* `waterpoint_type` - The kind of waterpoint
* `waterpoint_type_group` - The kind of waterpoint
* `population` - Population around the well

### Waterpoint Management
* `scheme_management` Who operates the waterpoint
* `scheme_name` - Who operates the waterpoint
* `permit` - If the waterpoint is permitted
* `management` - How the waterpoint is managed
* `management_group` - How the waterpoint is managed
* `payment` - What the water costs
* `payment_type` - What the water costs

### Waterpoint Installation
* `construction_year` - Year the waterpoint was constructed
* `funder` - Who funded the well
* `installer` - Organization that installed the well

### Data collection
* `date_recorded` - The date the row was entered
* `recorded_by` - Group entering this row of data

### Unknown
* `num_private` -
* `public_meeting` - True/False

## Inspecting Geographical Data

### Quantitative
The variables below provide geo-spatial coordinates for the waterpoint.
* `longitude` - GPS coordinate. Valid values fall in the interval [29.6, 40.4].
* `latitude` - GPS coordinate. Valid values fall in the interval [-11.7, -0.8].
* `gps_height` - Altitude of the well. Valid values fall in the interval [0, 5895] meters.
Below we map a random sample of 500 waterpoints.

In [ ]:
quantGeo = df[['longitude', 'latitude', 'gps_height', 'status_group']]
index_sample = list(np.random.choice(quantGeo.index, 500, replace=False))
sample = quantGeo.loc[index_sample]


In [ ]:
import folium

lat =  -6.3728
long = 34.8925

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=6)

In [ ]:
color_dict = {
    'functional' : 'green',
    'non functional': 'red',
    'functional needs repair': 'orange'
}

for index in sample.index:
        lat = sample['latitude'][index]
        long = sample['longitude'][index]
        status = str(sample['status_group'][index])
        color = color_dict[status]
        marker = folium.Circle(location=[lat, long], radius=1, popup=[lat, long], color=color)
        marker.add_to(base_map)

In [ ]:
base_map

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x='latitude', y='longitude', c='gps_height', data=quantGeo)
plt.colorbar()

#### Conclusions
There are a handful of waterpoints with bad `latitude` and `longitude`, there are also some waterpoints with negative `gps_height`.

### Qualitative 
The variables below provide a categorical description of waterpont location.
* `region` (`region_code`)  - Names (codes) for top-level administrative regions. There are 31 total [Link](https://en.wikipedia.org/wiki/Regions_of_Tanzania)
* `lga` (`district_code`) - Names (codes) for districts, which divide regions.
* `ward` - Names for wards, which divide districts.
* `subvillage` - Names for sub-villages, presumably these subdivide wards. 

In [ ]:
qualGeo = df[['region', 'region_code', 'lga', 'district_code', 'ward', 'subvillage', 'status_group', 'wpt_name']]
qualGeo.groupby(by=['region', 'region_code', 'lga', 'district_code', 'ward', 'subvillage', 'status_group']).count()

In [ ]:
qualGeo = df[['region', 'lga', 'ward', 'subvillage', 'wpt_name']]
qualGeo.groupby(by=['region', 'lga', 'ward', 'subvillage']).count()

### Conclusions
* Both `region_code` and `district_code` seem to be non-standard encodings and should probably be dropped.
* `lga` provides the names of districts.
* `ward` and `subvillage` have a huge number of values.

### Inspecting Water Table Data
* `basin` - Geographic water basin
* `water_quality` - The quality of the water
* `quality_group` - The quality of the water
* `quantity` - The quantity of water
* `quantity_group` - The quantity of water
* `source` - The source of the water
* `source_type` - The source of the water
* `source_class` - The source of the water

In [ ]:
waterQuality = df[['quality_group', 'water_quality', 'status_group', 'wpt_name']]
waterQuality.groupby(by=['quality_group', 'water_quality', 'status_group']).count()

In [ ]:
waterQuantity = df[['quantity_group', 'quantity', 'status_group', 'wpt_name']]
waterQuantity.groupby(by=['quantity_group', 'quantity', 'status_group']).count()

In [ ]:
df['basin'].value_counts()

In [ ]:
waterSource = df[['source_class', 'source_type', 'source', 'status_group', 'wpt_name']]
waterSource.groupby(by=['source_class', 'source_type', 'source', 'status_group']).count()

#### Conclusions
* `water_quality` provides information about whether a waterpoint is abandoned.
* `quantity` and `quantity_group` are duplicate columns.
* `source_class`, `source_type`, and `source` all provide essentially the same data. `source_class` seems like the best variable for initial investigation.

## Inspecting Waterpoint Data
* `wpt_name` - Name of the waterpoint if there is one
* `amount_tsh` - Total static head (amount water available to waterpoint)
* `extraction_type` - The kind of extraction the waterpoint uses
* `extraction_type_group` - The kind of extraction the waterpoint uses
* `extraction_type_class` - The kind of extraction the waterpoint uses
* `waterpoint_type` - The kind of waterpoint
* `waterpoint_type_group` - The kind of waterpoint
* `population` - Population around the well

In [ ]:
df['amount_tsh'].value_counts()

In [ ]:
df[['extraction_type_group', 'extraction_type_class', 'extraction_type']].value_counts()

In [ ]:
df[['waterpoint_type_group', 'waterpoint_type']].value_counts()

In [ ]:
df['population'].value_counts()

In [ ]:
df[df['population']>0]['population'].apply(lambda x: np.log(x)).hist()

#### Conclusions
* `amount_tsh` has a large number of zero values. It is hard to say if these represent missing values. Non-zero values would most likely benefit from log transform. 
* `extraction_type_class` seems to provide a good classification of pumps. Both `extraction_type_group` and `extraction_type` seem to be a mixture of models and types that is unlikely to provide much insight.
* `waterpoint_type_group` and `waterpoint_type` are essentially duplicate columns. I prefer the terminology used in `waterpoint_type_group`.
* `population` seems to use zero as a placeholder for missing values. Non-zero values might benefit from a log transformation.

## Inspecting Waterpoint Management Data
* `scheme_management` Who operates the waterpoint
* `scheme_name` - Who operates the waterpoint
* `permit` - If the waterpoint is permitted
* `management` - How the waterpoint is managed
* `management_group` - How the waterpoint is managed
* `payment` - What the water costs
* `payment_type` - What the water costs

In [ ]:
df[['scheme_management', 'scheme_name', 'status_group', 'wpt_name']].groupby(by=['scheme_management', 'scheme_name', 'status_group']).count()

In [ ]:
df['scheme_name'].value_counts()

In [ ]:
df[['management_group', 'management', 'status_group', 'wpt_name']].groupby(by=['management_group', 'management', 'status_group']).count()

In [ ]:
df[['payment_type', 'payment']].value_counts()

### Conclusions
* `scheme_name` seems to be describing the particular entity in charge of managing the waterpoint. This data may be overly granular.
* `scheme_management` describes the type of entity that manages the waterpoint.
* `management_group` and `management` seem to also describe management of the waterpoint. This representation seems to me better designed.
* `payment_type` and `payment` are duplicate columns with `payment` having the more descriptive labels.

## Inspecting Waterpoint Installation Data
* `construction_year` - Year the waterpoint was constructed
* `funder` - Who funded the well
* `installer` - Organization that installed the well

In [ ]:
df[['construction_year']].value_counts()

In [ ]:
df[['funder']].value_counts()

In [ ]:
df[['installer']].value_counts()

### Conclusions 
* zero seems to be a placeholder for missing values in the `construction_year` column.
* Both `installer` and `funder` have a huge number of labels. 

# Target class imbalance

In [ ]:
df['status_group'].value_counts(normalize=True)

# Overall Conclusions
* There are a small number of incorrect gps locations. These should be cleaned.
* the variables `region`, `lga`, `ward`, and `subvillage` provide a hierarchical description of the location of waterpoints.
* `region_code` and `district_code` should be dropped.
* `quantity_group` duplicates `quantity` and should be dropped.
* `quality_group` provides information about the quality of the water.
* `water_quality` could be engineered into a boolean `is_abandoned` variable.
* Keep `source_class`. Drop `source_type` and `source`.
* `amount_tsh` has a large number of zero values. It is hard to say if these represent missing values. Non-zero values would most likely benefit from log transform. 
* `extraction_type_class` seems to provide a good classification of pumps. Both `extraction_type_group` and `extraction_type` seem to be a mixture of pump models and types that is unlikely to provide much insight.
* `waterpoint_type_group` and `waterpoint_type` are essentially duplicate columns. I prefer the terminology used in `waterpoint_type_group`.
* `population` seems to use zero as a placeholder for missing values. Non-zero values might benefit from a log transformation.
* `scheme_name` seems to be describing the particular entity in charge of managing the waterpoint. This data may be overly granular.
* `scheme_management` describes the type of entity that manages the waterpoint.
* `management_group` and `management` seem to also describe management of the waterpoint. This representation seems to me better designed.
* `payment_type` and `payment` are duplicate columns with `payment` having the more descriptive labels.
* zero seems to be a placeholder for missing values in the `construction_year` column.
* Both `installer` and `funder` have a huge number of labels. 

## Drop
* `region_code`
* `district_code`
* `quantity_group`
* `water_quality` (or engineer into `is_abandoned`)
* `source_type`
* `source`
* `extraction_type_group`
* `extraction_type`
* `waterpoint_type`
* `scheme_name` (high dimensionality will require engineering otherwise)
* `scheme_management` (better represented in `management_group` and `management`)
* `payment_type`
* `installer` (high dimensionality will require engineering otherwise)
* `funder` (high dimensionality will require engineering otherwise)

